In [68]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch.autograd import Variable

# Define the custom dataset class
class PaddleDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.data.iloc[idx, :8], dtype=torch.float32)
        label = torch.tensor(self.data.iloc[idx, 8:].values.argmax(), dtype=torch.long)
        return features, label

# Define the neural network model
class PaddleClassifier(nn.Module):
    def __init__(self):
        super(PaddleClassifier, self).__init__()
        self.fc1 = nn.Linear(8, 32)
        self.fc2 = nn.Linear(32, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [69]:
# Load the data
data = pd.read_csv("TrainingDataReorg.tsv", sep="\t")
train_dataset = PaddleDataset(data)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [70]:
# Initialize the model, loss function, and optimizer
model = PaddleClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model_path = "paddle_classifier.pth"
onnx_path = "paddle_classifier.onnx"

# Load the model from disk if it exists
if os.path.isfile(model_path):
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    print("Model loaded from disk")

In [74]:
# Train the model
epochs = 100
for epoch in range(epochs):
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # if (epoch + 1) % 10 == 0:
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

print("Training complete")

Epoch [1/100], Loss: 0.6863
Epoch [2/100], Loss: 0.5225
Epoch [3/100], Loss: 0.3574
Epoch [4/100], Loss: 0.4642
Epoch [5/100], Loss: 0.5531
Epoch [6/100], Loss: 0.5222
Epoch [7/100], Loss: 0.6213
Epoch [8/100], Loss: 0.5658
Epoch [9/100], Loss: 0.5442
Epoch [10/100], Loss: 0.6045
Epoch [11/100], Loss: 0.6256
Epoch [12/100], Loss: 0.7128
Epoch [13/100], Loss: 0.4667
Epoch [14/100], Loss: 0.5820
Epoch [15/100], Loss: 0.5638
Epoch [16/100], Loss: 0.7075
Epoch [17/100], Loss: 0.6531
Epoch [18/100], Loss: 0.5269
Epoch [19/100], Loss: 0.2662
Epoch [20/100], Loss: 0.3859
Epoch [21/100], Loss: 0.5490
Epoch [22/100], Loss: 0.5721
Epoch [23/100], Loss: 0.5434
Epoch [24/100], Loss: 0.4796
Epoch [25/100], Loss: 0.3630
Epoch [26/100], Loss: 0.3348
Epoch [27/100], Loss: 0.4594
Epoch [28/100], Loss: 0.4884
Epoch [29/100], Loss: 0.5202
Epoch [30/100], Loss: 0.6482
Epoch [31/100], Loss: 0.4578
Epoch [32/100], Loss: 0.5793
Epoch [33/100], Loss: 0.6633
Epoch [34/100], Loss: 0.6025
Epoch [35/100], Loss: 0

In [77]:
# Save the trained model to disk
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Move the model back to CPU before exporting to ONNX
model.to("cpu")

# Export the model to ONNX format
dummy_input = Variable(torch.randn(1, 8))
torch.onnx.export(model, dummy_input, onnx_path, verbose=True, input_names=["input"], output_names=["output"])

# Check the ONNX model
#onnx_model = torch.onnx.load(onnx_path)
#onnx.checker.check_model(onnx_model)
print(f"ONNX model saved to {onnx_path}")

Model saved to paddle_classifier.pth
ONNX model saved to paddle_classifier.onnx


In [ ]:
#test model with set inputs
import numpy as np

def predict(model, input_data):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=0)
        predicted_class = torch.argmax(probabilities).item()
    return probabilities, predicted_class

# Example input data
input_data = [2.02, 28.22204, 0.4549797, 0, -0.377058, -0.2713, -0.4834, -3.15]  # Replace this with your own input data
probabilities, predicted_class = predict(model, input_data)

# Map the predicted_class index to the corresponding label
class_labels = ['MyPaddleIsRight', 'MyPaddleIsLeft', 'MyPaddleIsStill']
predicted_label = class_labels[predicted_class]

print(f"Probabilities: {probabilities.cpu().numpy()}")
print(f"Predicted class index: {predicted_class}")
print(f"Predicted class label: {predicted_label}")
